In [1]:
from utils import *
from model import *
from torch.utils.data import TensorDataset

In [20]:
with open("model_parameters.json","r") as f:
    model_parameters=json.load(f)

In [22]:
z = {**model_parameters, **data_parameters}

In [23]:
z

{'MODEL': 'roberta-base',
 'EPOCHS': 15,
 'BATCH_SIZE': 32,
 'EPS': 1e-06,
 'LEARNING_RATE': 1e-06,
 'GRADIENT_CLIP': 1.0,
 'HIDDEN_UNIT1': 128,
 'HIDDEN_UNIT2': None,
 'FREEZE_BERT': False,
 'SEED': 31,
 'TRAINPATH': '../data/subtask3-coreference/train.json',
 'DEVPATH': '../data/subtask3-coreference/dev.json',
 'TESTPATH': '../data/subtask3-coreference/test.json',
 'TRAIN_CROSS': True,
 'TRAIN_RATIO': 0.25,
 'DEV_CROSS': True,
 'DEV_RATIO': 0.25,
 'TEST_CROSS': False,
 'TEST_RATIO': None}

In [2]:
data_parameters = {"SEED":31,
                   "TRAINPATH": "../data/subtask3-coreference/train.json",
                   "DEVPATH": "../data/subtask3-coreference/dev.json",
                   "TESTPATH": "../data/subtask3-coreference/test.json",
                   "TRAIN_CROSS": True, "TRAIN_RATIO": 0.25,
                   "DEV_CROSS" :  True,"DEV_RATIO":   0.25,
                   "TEST_CROSS":  False,"TEST_RATIO":  None,              
                   "MODEL": "roberta-base"}

In [18]:
model_parameters = {"MODEL": "roberta-base",
                    "EPOCHS"       : 15,
                    "BATCH_SIZE"   : 32,
                    "EPS"          : 1e-6,
                    "LEARNING_RATE": 1e-6,
                    "GRADIENT_CLIP": 1.0,
                    "HIDDEN_UNIT1" : 128,
                    "HIDDEN_UNIT2" : None,
                    "FREEZE_BERT"  : False}

In [4]:
TR_II,TR_AM,TR_LABEL,DE_II,DE_AM,DE_LABEL,TE_II,TE_AM,TE_LABEL = gimme_datasets(**data_parameters)

In [5]:
TR_II.shape,DE_II.shape,TE_II.shape

(torch.Size([13268, 256]), torch.Size([2084, 256]), torch.Size([782, 256]))

### Model

In [6]:
NEPTUNE_API = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiOGQ0YWZhNjAtYWRmOS00ZWYyLWI4NGYtMWFhZmI1MDI2YTk3In0="

In [7]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='models/checkpoints/',
    filename=model_parameters["MODEL"]+'-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min',
)
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.00,
   patience=3,
   verbose=False,
   mode='min')

In [8]:
model = CorefClassifier(MODEL       = model_parameters["MODEL"],
                        TRAIN_DATA  =TensorDataset(TR_II,TR_AM,TR_LABEL),
                        DEV_DATA    =TensorDataset(DE_II,DE_AM,DE_LABEL),
                        HIDDEN_UNIT1= model_parameters["HIDDEN_UNIT1"],
                        BATCH_SIZE  = model_parameters["BATCH_SIZE"],
                        LR          = model_parameters["LEARNING_RATE"],
                        EPS         = model_parameters["EPS"],
                        EPOCHS      = model_parameters["EPOCHS"],
                        FREEZE_BERT = model_parameters["FREEZE_BERT"])

In [9]:
neptune_logger = NeptuneLogger(api_key=NEPTUNE_API,
                               project_name="fatihbeyhan/CASE21-SUBTASK3",
                               )

NeptuneLogger will work in online mode


In [13]:
trainer = Trainer(max_epochs = model_parameters["EPOCHS"],
                  #gpus = ,
                  #gradient_clip_val= GRADIENT_CLIP,
                  logger=neptune_logger,
                  callbacks = [checkpoint_callback,
                               early_stop_callback]
                 )

GPU available: True, used: False
TPU available: None, using: 0 TPU cores
/home/fatihbeyhan/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)


In [11]:
#train_data = TensorDataset(TR_II,TR_AM,TR_LABEL)
#dev_data = TensorDataset(DE_II,DE_AM,DE_LABEL)
#test_data = TensorDataset(TE_II,TE_AM,TE_LABEL)

In [14]:
trainer.fit(model)


  | Name          | Type             | Params
---------------------------------------------------
0 | model         | RobertaModel     | 124 M 
1 | hidden_layer1 | Linear           | 98.4 K
2 | hidden_layer2 | Linear           | 129   
3 | lossfn        | BCELoss          | 0     
4 | valid_metrics | MetricCollection | 0     
---------------------------------------------------
124 M     Trainable params
0         Non-trainable params
124 M     Total params
498.977   Total estimated model params size (MB)


/home/fatihbeyhan/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


MisconfigurationException: ModelCheckpoint(monitor='val_loss') not found in the returned metrics: ['train_loss']. HINT: Did you call self.log('val_loss', tensor) in the LightningModule?